Prometheus
----------

![](https://prometheus.io/assets/architecture.png)

Quelle: [Prometheus](https://prometheus.io) 

- - -

[Prometheus](https://prometheus.io) ist ein Open-Source-Toolkit zur Systemüberwachung und -Alamierung, das ursprünglich bei SoundCloud entwickelt wurde.

Prometheus trat der [Cloud Native Computing Foundation](https://www.cncf.io/) im Jahr 2016 als zweites gehostetes Projekt nach Kubernetes bei.

Zusammen mit zwei weiteren Produkten ist der Monitoring Open Source Stack wie folgt:

* **Prometheus** ist ein Zeitreihendatenbank- (time series database) und Überwachungstool, mit dem Metrik Endpunkte abgefragt und die von diesen Endpunkten bereitgestellten Daten abgeholt und verarbeitet werden. Daten können mit PromQL abgefragt werden.
* **Grafana** ist ein Datenvisualisierungs- und Analysetool, mit dem Sie Dashboards und Diagramme für Ihre Metrikdaten erstellen können.
* **Alertmanager**, der normalerweise zusammen mit Prometheus bereitgestellt wird, bildet die Warnschicht des Stapels. Er verarbeitet von Prometheus generierte Warnungen und dedupliziert (eliminieren von redundanten Daten), gruppiert und leitet sie z.B. via E-Mail weiter.


***
### Installation

Prometheus ist am einfachsten mittels [helm](https://helm.io) installierbar.


In [ ]:
%%bash
kubectl create namespace monitoring
helm repo add prometheus-community https://prometheus-community.github.io/helm-charts
helm repo update
helm install monitoring prometheus-community/kube-prometheus-stack -n monitoring

Die Ports der User Interfaces

* Grafana (Service: prometheus-operator-grafana) 
* Prometheus (Service: prometheus-operator-prometheus)
* Alertmanager (Service: prometheus-operator-alertmanager)

sind nur innerhalb des Cluster zugänglich. Deshalb müssen wir diese zuerst freischalten bzw. gegen Aussen öffnen:

In [ ]:
! kubectl -n monitoring patch service monitoring-grafana                      -p '{"spec": {"type": "LoadBalancer"}}'
! kubectl -n monitoring patch service monitoring-kube-prometheus-prometheus   -p '{"spec": {"type": "LoadBalancer"}}'
! kubectl -n monitoring patch service monitoring-kube-prometheus-alertmanager -p '{"spec": {"type": "LoadBalancer"}}'

Sind Sie freigeschaltet, können wir Sie wie andere Services via Cluster-IP und Port erreichen:

In [ ]:
! echo "Grafana UI     : " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-grafana -o=jsonpath='{ .spec.ports[0].nodePort }')
! echo "Prometheus UI  : " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-kube-prometheus-prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')
! echo "Alertmanager UI: " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-kube-prometheus-alertmanager -o=jsonpath='{ .spec.ports[0].nodePort }')

**Grafana UI** Einloggen mittels User/Password **admin/prom-operator**. Auf der linken Seite kann zwischen einer Reihe von vorbereitenden Dashboards ausgewählt werden, z.B. Nodes.

In der **Prometheus** Oberfläche kann mittels der Abfragesprache [PromQL](https://prometheus.io/docs/prometheus/latest/querying/basics/) gezielt Ressourcen ausgewählt werden, z.B. durch Query von `apiserver_storage_objects`.

Der **Alertmanager** dient zum Verarbeiten von Warnungen. Für ein Beispiel siehe [Notifikation Beispiel](https://prometheus.io/docs/alerting/notification_examples/)

- - -
### Custom Ressourcen

Der Prometheus [Operator](https://github.com/coreos/prometheus-operator/blob/master/README.md) stellt eine Reihe von [benutzerdefinierten Ressourcen (CRDs)](https://kubernetes.io/docs/tasks/access-kubernetes-api/extend-api-custom-resource-definitions/) für das Monitoring bereit.

* **Prometheus**, für eine Prometheus Instanz.
* **Alertmanager**, für eine Alertmanager Instanz
* **ServiceMonitor**, für die Überwachung von Kubernetes-Diensten.
* **PodMonitor**, für die Überwachung von Pods.


In [ ]:
! kubectl api-resources | grep monitoring
! echo "---------------------------------"
! kubectl explain servicemonitors                   